# Depth Estimation using Stereo Vision

<div>
<iframe src="https://slides.com/naresh-ub/cvip-lec-6-7-8/embed" width="100%" height="500" title="Test Title" scrolling="no" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe>
</div>

## Depth estimation (openCV implementation)

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import imageio.v3 as iio
from io import BytesIO
from pyodide.http import pyfetch
import asyncio

async def read_image_from_url(url: str) -> np.ndarray:
    """
    Fetches an image from a URL using pyfetch and decodes it with imageio.v3.
    """
    response = await pyfetch(url=url, method="GET")
    data = BytesIO(await response.bytes())
    img = iio.imread(data, index=None)
    return img

async def compute_and_show_depth(
    url_left: str,
    url_right: str,
    num_disparities: int = 16,
    block_size: int = 15
) -> np.ndarray:
    """
    Reads left/right stereo images from URLs, shows left, right, and difference images,
    then computes & displays a disparity (depth) map using OpenCV StereoBM.
    """
    # Read images
    imgL = await read_image_from_url(url_left)
    imgR = await read_image_from_url(url_right)

    # Convert to grayscale if needed
    if imgL.ndim == 3:
        imgL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    if imgR.ndim == 3:
        imgR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Compute difference image
    diff = cv.absdiff(imgL, imgR)

    # Display left, right, and difference
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(imgL, cmap='gray')
    plt.title('Left Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(imgR, cmap='gray')
    plt.title('Right Image')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(diff, cmap='gray')
    plt.title('Difference')
    plt.axis('off')
    plt.show()

    # Ensure num_disparities is a multiple of 16
    if num_disparities % 16 != 0:
        num_disparities = ((num_disparities // 16) + 1) * 16

    # Compute disparity map
    stereo = cv.StereoBM_create(numDisparities=num_disparities,
                                blockSize=block_size)
    disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

    # Display disparity (depth) map
    plt.figure(figsize=(8, 6))
    plt.imshow(disp, cmap='gray')
    plt.title("Disparity (Depth) Map")
    plt.axis('off')
    plt.colorbar(label='Disparity')
    plt.show()

    return disp

# Example usage
async def main():
    LEFT_URL  = "https://raw.githubusercontent.com/canberkgurel/DisparityMapfromStereoPair/master/tsukuba_l.png"
    RIGHT_URL = "https://raw.githubusercontent.com/canberkgurel/DisparityMapfromStereoPair/master/tsukuba_r.png"
    await compute_and_show_depth(LEFT_URL, RIGHT_URL, num_disparities=16, block_size=15)

# Run it
await main()

## From scratch implementation of depth estimation

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import imageio.v3 as iio
from io import BytesIO
from pyodide.http import pyfetch
import asyncio

async def read_image_from_url(url: str) -> np.ndarray:
    resp = await pyfetch(url=url, method="GET")
    data = BytesIO(await resp.bytes())
    return iio.imread(data, index=None)

def detect_corners(img: np.ndarray,
                   max_corners: int = 500,
                   quality: float = 0.01,
                   min_dist: float = 10.0) -> np.ndarray:
    corners = cv.goodFeaturesToTrack(img, max_corners, quality, min_dist)
    return corners.reshape(-1, 2)

def extract_patches(img: np.ndarray,
                    pts: np.ndarray,
                    patch_size: int = 21) -> tuple[np.ndarray, np.ndarray]:
    half = patch_size // 2
    h, w = img.shape
    patches, valid = [], []
    for x, y in pts.astype(int):
        if y-half<0 or y+half>=h or x-half<0 or x+half>=w:
            continue
        patches.append(img[y-half:y+half+1, x-half:x+half+1].flatten())
        valid.append((x, y))
    return np.array(patches), np.array(valid)

def match_patches(p1, pts1, p2, pts2, max_ssd: float = 1e6):
    m1, m2 = [], []
    for i,d1 in enumerate(p1):
        errs = np.sum((p2 - d1)**2, axis=1)
        j = np.argmin(errs)
        if errs[j] < max_ssd:
            m1.append(pts1[i]);  m2.append(pts2[j])
    return np.array(m1), np.array(m2)

def normalize_points(pts):
    mean = pts.mean(axis=0)
    s = np.sqrt(2)/pts.std()
    T = np.array([[s,0,-s*mean[0]],[0,s,-s*mean[1]],[0,0,1]])
    ph = np.hstack([pts, np.ones((len(pts),1))])
    pn = (T @ ph.T).T
    return pn[:,:2], T

def eight_point_algorithm(pts1, pts2):
    p1n, T1 = normalize_points(pts1)
    p2n, T2 = normalize_points(pts2)
    A = np.zeros((len(pts1), 9))
    for i,(x1,y1) in enumerate(p1n):
        x2,y2 = p2n[i]
        A[i] = [x1*x2, x1*y2, x1,
                y1*x2, y1*y2, y1,
                x2,    y2,    1]
    _,_,Vt = np.linalg.svd(A)
    F = Vt[-1].reshape(3,3)
    U,S,Vt2 = np.linalg.svd(F);  S[2]=0
    F = U @ np.diag(S) @ Vt2
    F = T2.T @ F @ T1
    return F / F[2,2]

def compute_fundamental_ransac(pts1, pts2, iters=500, tol=1e-3):
    best_in, bestF = [], None
    N = len(pts1)
    h1 = np.hstack([pts1, np.ones((N,1))])
    h2 = np.hstack([pts2, np.ones((N,1))])
    for _ in range(iters):
        idx = np.random.choice(N, 8, replace=False)
        Fc = eight_point_algorithm(pts1[idx], pts2[idx])
        l2 = (Fc @ h1.T).T
        l1 = (Fc.T @ h2.T).T
        num   = np.sum(h2*(Fc @ h1.T).T, axis=1)**2
        denom = l2[:,0]**2 + l2[:,1]**2 + l1[:,0]**2 + l1[:,1]**2
        errs  = num/denom
        inliers = np.where(errs<tol)[0]
        if len(inliers)>len(best_in):
            best_in, bestF = inliers, Fc
    if bestF is None:
        raise RuntimeError("RANSAC failed")
    Fref = eight_point_algorithm(pts1[best_in], pts2[best_in])
    mask = np.zeros(N, bool);  mask[best_in]=True
    return Fref / Fref[2,2], mask

def draw_epipolar_lines(img1, img2, F, pts1, pts2, mask, max_lines=10):
    in1 = pts1[mask];  in2 = pts2[mask]
    k = min(len(in1), max_lines)
    in1, in2 = in1[:k], in2[:k]
    l1 = (F.T @ np.hstack([in2, np.ones((k,1))]).T).T
    l2 = (F   @ np.hstack([in1, np.ones((k,1))]).T).T

    fig, axes = plt.subplots(1,2, figsize=(12,5))
    for ax, img, lines, pts, col in zip(
        axes, [img1,img2], [l1,l2], [in1,in2], ['r','b']
    ):
        ax.imshow(img, cmap='gray')
        h,w = img.shape
        for (a,b,c),(x,y) in zip(lines, pts):
            x0,x1 = 0,w
            y0 = -(a*x0 + c)/b;  y1 = -(a*x1 + c)/b
            ax.plot([x0,x1],[y0,y1], col)
            ax.scatter(x, y, c=col, s=30)
        ax.axis('off')
    plt.suptitle("Epipolar Lines")
    plt.show()

def compute_dense_disparity_fast(imgL, imgR, max_disp=64, block_size=9):
    h,w = imgL.shape
    disp = np.zeros((h,w), np.float32)
    best = np.full((h,w), np.inf, np.float32)
    for d in range(max_disp):
        shifted = np.zeros_like(imgR)
        shifted[:, d:] = imgR[:, :w-d]
        cost = (imgL.astype(np.float32)-shifted.astype(np.float32))**2
        cost = cv.boxFilter(cost, -1, (block_size,block_size), normalize=False)
        m = cost < best
        disp[m]=d;  best[m]=cost[m]
    return disp

async def main():
    LEFT = "https://raw.githubusercontent.com/canberkgurel/DisparityMapfromStereoPair/master/tsukuba_l.png"
    RIGHT= "https://raw.githubusercontent.com/canberkgurel/DisparityMapfromStereoPair/master/tsukuba_r.png"

    imgL = await read_image_from_url(LEFT)
    imgR = await read_image_from_url(RIGHT)
    if imgL.ndim==3: imgL=cv.cvtColor(imgL,cv.COLOR_BGR2GRAY)
    if imgR.ndim==3: imgR=cv.cvtColor(imgR,cv.COLOR_BGR2GRAY)

    # show stereo pair
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1); plt.imshow(imgL, cmap='gray'); plt.title("Left");  plt.axis('off')
    plt.subplot(1,2,2); plt.imshow(imgR, cmap='gray'); plt.title("Right"); plt.axis('off')
    plt.show()

    # sparse features + F
    cL=detect_corners(imgL); cR=detect_corners(imgR)
    pL,ptsL=extract_patches(imgL,cL); pR,ptsR=extract_patches(imgR,cR)
    pts1,pts2=match_patches(pL,ptsL,pR,ptsR)
    F,mask=compute_fundamental_ransac(pts1,pts2)
    print("F:",F)

    # epipolar viz
    draw_epipolar_lines(imgL, imgR, F, pts1, pts2, mask)

    # dense disparity comparison
    disp_custom = compute_dense_disparity_fast(imgL, imgR, max_disp=64, block_size=9)
    stereo = cv.StereoBM_create(numDisparities=64, blockSize=15)
    disp_cv = stereo.compute(imgL, imgR).astype(np.float32)/16.0

    diff = np.abs(disp_cv - disp_custom)
    fig, axs = plt.subplots(1,3, figsize=(18,6))
    for ax, mat, title in zip(axs, [disp_cv, disp_custom, diff],
                              ["OpenCV", "Custom", "Abs Diff"]):
        im = ax.imshow(mat, cmap='jet'); ax.set_title(title); ax.axis('off')
        plt.colorbar(im, ax=ax, fraction=0.046)
    plt.suptitle("Disparity Comparison"); plt.show()

    print("MAE:", np.mean(diff), "RMSE:", np.sqrt(np.mean(diff**2)))

await main()